In [ ]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import os
import random
import matplotlib.pyplot as plt
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
                                              # class_mode='categorical')

TRAINING_DIR = "/content/drive/MyDrive/data/train"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      width_shift_range=0.11,
      height_shift_range=0.11,
      zoom_range=0.1,
      shear_range=0.1,
      horizontal_flip=True,
      rotation_range=11,
      fill_mode='nearest', validation_split=0.001
      )


train_generator = training_datagen.flow_from_directory(
	directory = TRAINING_DIR,
	target_size=(224, 224),
	class_mode='categorical',
  batch_size=32,
  subset='training',
  seed = 1
)
validation_generator = training_datagen.flow_from_directory(batch_size=32,
                                                 directory= TRAINING_DIR,
                                                 shuffle=True,
                                                 target_size=(224, 224),
                                                 subset="validation",
                                                 class_mode='categorical')


Found 15399 images belonging to 21 classes.
Found 0 images belonging to 21 classes.


In [ ]:
if len(validation_generator) > 0:
    print("Data found in the generator.")
    print(f"Number of batches: {len(validation_generator)}")
    print(f"Number of samples: {validation_generator.samples}")
else:
    print("No data found in the generator.")

No data found in the generator.


In [ ]:
pre_trained_model = tf.keras.applications.MobileNetV2(input_shape = (224, 224, 3),
                                include_top = False,
                                weights = "imagenet")



9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
print(pre_trained_model.layers)

[<keras.src.engine.input_layer.InputLayer object at 0x7ac74f7ef130>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ac74da00a00>, <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7ac74da01b40>, <keras.src.layers.activation.relu.ReLU object at 0x7ac74da01de0>, <keras.src.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x7ac74da02380>, <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7ac74da03700>, <keras.src.layers.activation.relu.ReLU object at 0x7ac74da03e80>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ac74da03250>, <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7ac74c341c00>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ac74c341f30>, <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7ac74c3432b0>, <keras.src.layers.activation.relu.ReLU object at 0x7ac74c343490>, <keras.src.layers.reshaping.ze

In [ ]:
pre_trained_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [ ]:

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model


x = tf.keras.layers.GlobalAveragePooling2D()(pre_trained_model.output)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

x = tf.keras.layers.Dense(21, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=6, min_lr=0.0000005)


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/my_models/best_model1231.keras", monitor='accuracy', save_best_only=True, mode='auto')

In [ ]:

model.compile(loss = "categorical_crossentropy", optimizer="nadam", metrics=['accuracy'])
history = model.fit(train_generator,
validation_data=(validation_generator),
epochs=100,batch_size = 16, callbacks=[reduce_lr,checkpoint])

Epoch 1/100
434/434 [==============================] - 3183s 7s/step - loss: 1.4914 - accuracy: 0.5487 - val_loss: 7.2029 - val_accuracy: 0.2038 - lr: 0.0010
Epoch 2/100
434/434 [==============================] - 256s 589ms/step - loss: 1.0668 - accuracy: 0.6626 - val_loss: 18.7074 - val_accuracy: 0.0745 - lr: 0.0010
Epoch 3/100
434/434 [==============================] - 252s 579ms/step - loss: 0.9252 - accuracy: 0.6999 - val_loss: 9.8085 - val_accuracy: 0.1535 - lr: 0.0010
Epoch 4/100
434/434 [==============================] - 252s 580ms/step - loss: 0.8400 - accuracy: 0.7275 - val_loss: 6.8386 - val_accuracy: 0.2593 - lr: 0.0010
Epoch 5/100
434/434 [==============================] - 248s 571ms/step - loss: 0.7859 - accuracy: 0.7443 - val_loss: 3.1145 - val_accuracy: 0.3599 - lr: 0.0010
Epoch 6/100
434/434 [==============================] - 248s 572ms/step - loss: 0.7347 - accuracy: 0.7599 - val_loss: 2.2728 - val_accuracy: 0.4892 - lr: 0.0010
Epoch 7/100
434/434 [====================

In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/my_models/best_model.keras")

In [ ]:
loaded_model.save_weights('/content/drive/MyDrive/data/weights.h5')

In [ ]:
entries = os.listdir("/content/drive/MyDrive/data/train")
entries = np.array(entries)
entries

array(['button', 'floating_action_button', 'data_table', 'dropdown_menu',
       'alert', 'checkbox_checked', 'chip', 'grid_list',
       'checkbox_unchecked', 'card', 'radio_button_unchecked', 'slider',
       'text_field', 'switch_enabled', 'radio_button_checked',
       'text_area', 'image', 'switch_disabled', 'label', 'menu',
       'tooltip'], dtype='<U22')

In [ ]:
initial_learning_rate = 0.0000001
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

loaded_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = loaded_model.fit(train_generator,
epochs=100,batch_size = 16, callbacks=[reduce_lr,checkpoint])

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing import image
df = pd.read_csv("/content/drive/MyDrive/data/predictions_example.csv")

predict = []
for index, row in df.iterrows():
  filename = row['filename']
  img = image.load_img("/content/drive/MyDrive/data/test/" + filename, target_size=(224, 224))
  img_array = image.img_to_array(img)
  img_array /= 255.0
  predict.append(loaded_model.predict(np.array([img_array])))

predict = np.array(predict)
predictions = predict
df['predicted_class'] = np.array(list(train_generator.class_indices.keys()))[np.argmax(predictions, axis = 1)]

Показано результат, скорочений до останніх рядків (5000).
1/1 [==============================] - 0s 55ms/step
3476
(224, 224, 3)
1/1 [==============================] - 0s 87ms/step
3477
(224, 224, 3)
1/1 [==============================] - 0s 90ms/step
3478
(224, 224, 3)
1/1 [==============================] - 0s 83ms/step
3479
(224, 224, 3)
1/1 [==============================] - 0s 85ms/step
3480
(224, 224, 3)
1/1 [==============================] - 0s 98ms/step
3481
(224, 224, 3)
1/1 [==============================] - 0s 95ms/step
3482
(224, 224, 3)
1/1 [==============================] - 0s 57ms/step
3483
(224, 224, 3)
1/1 [==============================] - 0s 54ms/step
3484
(224, 224, 3)
1/1 [==============================] - 0s 52ms/step
3485
(224, 224, 3)
1/1 [==============================] - 0s 65ms/step
3486
(224, 224, 3)
1/1 [==============================] - 0s 51ms/step
3487
(224, 224, 3)
1/1 [==============================] - 0s 51ms/step
3488
(224, 224, 3)
1/1 [==============

In [ ]:
del df["Target"]

In [ ]:
df['predicted_class'] = np.array(list(train_generator.class_indices.keys()))[np.argmax(predictions, axis = 2)]

In [ ]:
df

,ID,predicted_class
0,079c715e-0387-4fa2-abc0-5cc7a81120f6.jpg,grid_list
1,a7b15b95-c998-4c3a-85c9-9eae9143c894.jpg,text_field
2,69123d68-e442-4cae-8f86-9c3f4c985760.jpg,tooltip
3,c61abe0a-54bc-4cfd-b5bd-ccfff2e0d535.jpg,text_area
4,ec5c04b8-d75f-4414-821f-6e845a6bfad3.jpg,button
...,...,...
5138,deecda16-bb84-4164-8857-8c1ddfca5e61.jpg,floating_action_button
5139,da48f7f4-bec7-4ad1-bebf-1c7a7de667f4.jpg,alert
5140,e82902f5-5b77-4ed9-91a4-d20300aef23f.jpg,radio_button_checked
5141,e9a52c92-865a-45aa-a6ba-ea427b462f48.jpg,grid_list


In [ ]:
csv_file_path = '/content/drive/MyDrive/my_models/output_file1.csv'
df.columns = ["ID", "Target"]
df.to_csv(csv_file_path, index=False)
